<a href="https://colab.research.google.com/github/Mike030668/Rotation_tran_Text2Image_2_task_Tex2Video/blob/main/train/train_baseline_with__R_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Mike030668/Rotation_tran_Text2Image_2_task_Tex2Video.git -q
!pip install --upgrade --no-cache-dir gdown -q

In [2]:
import torch
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

MAIN_DIR =  "/content/Rotation_tran_Text2Image_2_task_Tex2Video"
import sys
sys.path.append(MAIN_DIR)

import numpy as np
from torch import nn
import random
# отрисовка прохождения цикла
from tqdm.notebook import tqdm
import time

from utills.step_points import ComputeDiffPoints, Shuff_Reshuff
from utills.rotations import RotationVectors
from models.R_baseline import R_SpliterSimple
from utills.clearing import flush_memory
from utills.utills import optimizer_to
from utills.trainers import R_trainer
from models.constants import EMB_DIM, MAX_DELTA, WAYS

import warnings
warnings.filterwarnings('ignore')

print(f'device is {DEVICE}\n')
torch.cuda.memory_summary(device=DEVICE, abbreviated=False)

device is cuda



'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |\n|---------------------------------------------------------------------------|\n| Active memory         |      0 B   |      0 B   |      0 B   |      0 B   |\n|       from large pool |      0 B   |      0 B   |

# seed freeze

In [3]:
seed = 10

random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# Load tensor data

In [4]:
!gdown --id "1msbazrYX8KgxHzVaU7WwGOFUl5GM3xjk"
!unzip /content/bothway_embed_data_min_300.zip
all_data = torch.load("/content/bothway_embed_data_min_300.pth")

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1msbazrYX8KgxHzVaU7WwGOFUl5GM3xjk
From (redirected): https://drive.google.com/uc?id=1msbazrYX8KgxHzVaU7WwGOFUl5GM3xjk&confirm=t&uuid=e9c1c7e3-f2b8-4535-b8c6-3c268cd28160
To: /content/bothway_embed_data_min_300.zip
100% 129M/129M [00:01<00:00, 95.4MB/s]
Archive:  /content/bothway_embed_data_min_300.zip
  inflating: bothway_embed_data_min_300.pth  


In [5]:
all_data[0]

{'id_movi': 47537,
 'img_embeds': [tensor([[-0.4927,  1.2588, -0.3252,  ..., -0.0650,  1.2676, -0.1646]],
         dtype=torch.float16),
  tensor([[-0.3416,  1.3457, -0.3796,  ..., -0.0455,  1.6543,  0.0765]],
         dtype=torch.float16),
  tensor([[-0.3682,  1.3066, -0.3611,  ..., -0.1733,  1.5977,  0.0778]],
         dtype=torch.float16),
  tensor([[-0.4695,  1.1553, -0.2303,  ...,  0.0641,  1.5146,  0.0908]],
         dtype=torch.float16),
  tensor([[-0.4551,  1.0469, -0.2368,  ..., -0.4043,  1.3037, -0.0432]],
         dtype=torch.float16),
  tensor([[-0.2996,  1.1777, -0.1384,  ..., -0.2064,  1.4854,  0.1617]],
         dtype=torch.float16),
  tensor([[-0.3345,  1.2402, -0.0933,  ..., -0.2900,  1.5459, -0.0803]],
         dtype=torch.float16),
  tensor([[-0.6611,  1.1943, -0.1829,  ..., -0.1976,  1.1055, -0.2615]],
         dtype=torch.float16),
  tensor([[-0.5220,  0.8721, -0.3948,  ..., -0.0234,  1.7588,  0.0391]],
         dtype=torch.float16),
  tensor([[-0.3567,  1.1631, -0

# Both ways steps base spliter

In [6]:
model = R_SpliterSimple(max_delta_time = MAX_DELTA,
                        emb_dim = EMB_DIM,
                        ways = WAYS
                        )


In [7]:
total_params = sum(p.numel() for p in model.parameters())
print(f"total_params {total_params}\n")

#If you want to calculate only the trainable parameters:
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"trainable_params {trainable_params}")


total_params 1791657

trainable_params 1791657


# Rotation train R_SpliterSimple

In [8]:
BASE_PATH = "/content/drive/Othercomputers/My_comp/Документы/МФТИ_Наука_о_данных/Diplom_project"
PATH_dir = BASE_PATH + '/Spliters/base_R_spliter'

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-03)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

# to(DEVICE)
model.to(DEVICE)
optimizer_to(optimizer, DEVICE)

# inite ComputeDiffPoints class
cdp = ComputeDiffPoints()

RT = R_trainer(
                model,
                DEVICE,
                path_save = PATH_dir,
                next_train = False,
                )


## start training

In [9]:
RT.trailoop(
            train_data = all_data,
            max_batch  = 15,
            optimizer = optimizer,
            scheduler = scheduler,
            maker_points = cdp,            # class for calculation point for normal, back, rotation and diff trainning
            rotator = RotationVectors,     # special class of rotation vmatrix of vectots
            epochs = 7,                    # q-ty epochs to train
            friq_save_checkpoint = 10,     # friquence of saving checkpoints
            jdun_train = 4,                # wait without good changes and reduce exp_rate
            jdun_end = 4,                  # reducing exp_rate without good profit in train again
            use_on_epoch = 0.3,            # random part of data using on epoch
            exp_rate = 0.95,               # reducing exp rate if do not good fithess in current velocity of reduction
            window = 2,                    # window back of loss to update best
            learning_rate = 1e-03,         # special Lr to start
            update_lr = True,              # if use last chkpt, but want to start with special Lr
            reset_losses_acc = True,       # if need start as new
            add_back_train = True,         # use movi from end too
            suff_direct =  True,           # suff directions when add_back_train
            add_rote_train = True,
            add_diff_train = True,
            )

Пробегаемся по всем эпохам:   0%|          | 0/7 [00:00<?, ?EPOHS/s]

Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 0 all_loss 3.65855 | acc 0.92354 | mse_loss 3.27625 | cos_loss 0.38255 | lr 1.00000e-03 



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 1 all_loss 3.51016 | acc 0.86616 | mse_loss 3.12518 | cos_loss 0.38518 | lr 9.00000e-04 



...

Ep 4 all_loss 3.13006 | acc 0.78677 | mse_loss 2.77126 | cos_loss 0.35893 | lr 6.56100e-04 - save_acc - save_mse - save_cos - save_loss 



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 5 all_loss 3.04635 | acc 0.75854 | mse_loss 2.71128 | cos_loss 0.33521 | lr 5.90490e-04 - save_mse - save_cos - save_loss 



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Movi_loss.backward on 49 step 
Train is finished, saved the best model from 5_ep
Ep 6 all_loss 3.06510 | acc 0.76129 | mse_loss 2.72509 | cos_loss 0.33985 | lr 5.31441e-04 wait_-1 - load best_loss_model 5 ep



## training from last ckpt as new train

In [10]:
BASE_PATH = "/content/drive/Othercomputers/My_comp/Документы/МФТИ_Наука_о_данных/Diplom_project"
PATH_dir = BASE_PATH + '/Spliters/base_R_spliter'

last_cpt = PATH_dir+'/R_SpliterSimple_loss_5.cpt'
# inite optimizer and scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-03)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

# to(DEVICE)
model.to(DEVICE)
optimizer_to(optimizer, DEVICE)

# inite ComputeDiffPoints class
cdp = ComputeDiffPoints()

RT = R_trainer(
                model,
                DEVICE,
                path_save = PATH_dir,
                next_train = True,
                path_cpt = last_cpt
                )


In [ ]:
RT.trailoop(
            train_data = all_data,
            max_batch  = 15,
            optimizer = optimizer,
            scheduler = scheduler,
            maker_points = cdp,            # class for calculation point for normal, back, rotation and diff trainning
            rotator = RotationVectors,     # special class of rotation vmatrix of vectots
            epochs = 70,                   # q-ty epochs to train
            friq_save_checkpoint = 10,     # friquence of saving checkpoints
            jdun_train = 4,                # wait without good changes and reduce exp_rate
            jdun_end = 4,                  # reducing exp_rate without good profit in train again
            use_on_epoch = 0.3,            # random part of data using on epoch
            exp_rate = 0.95,               # reducing exp rate if do not good fithess in current velocity of reduction
            window = 2,                    # window back of loss to update best
            learning_rate = 1e-03,         # special Lr to start
            update_lr = True,              # if use last chkpt, but want to start with special Lr
            reset_losses_acc = True,       # if need start as new
            add_back_train = True,         # use movi from end too
            suff_direct =  True,           # suff directions when add_back_train
            add_rote_train = True,
            add_diff_train = False,
            )


Пробегаемся по всем эпохам:   0%|          | 0/70 [00:00<?, ?EPOHS/s]

Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 0 all_loss 2.36757 | acc 0.76688 | mse_loss 2.11568 | cos_loss 0.25198 | lr 1.00000e-03 



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 1 all_loss 2.27208 | acc 0.75979 | mse_loss 2.02287 | cos_loss 0.24922 | lr 9.00000e-04 



...

Ep 10 all_loss 1.67886 | acc 0.77097 | mse_loss 1.45085 | cos_loss 0.22800 | lr 3.48678e-04 - save_cos - save_10_ep_cpt  - save best_cos_model 10 ep checkpoint



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Ep 11 all_loss 1.66616 | acc 0.77170 | mse_loss 1.44031 | cos_loss 0.22584 | lr 3.13811e-04 - save_mse - save_cos - save_loss 



Пробегаемся по всем фильмам: 0 movi  [00:00, ? movi /s]

Movi 28746 step 100 base_way mse 1.38679 | cos 0.22257 | lr 2.82430e-04